In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta

## Data Understanding

In [2]:
# Load the dataset
dataset = pd.read_csv('order_brush_order.csv')

In [3]:
# Show sample dataset
dataset.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [4]:
# Show more information about the dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   orderid     222750 non-null  int64 
 1   shopid      222750 non-null  int64 
 2   userid      222750 non-null  int64 
 3   event_time  222750 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


In [5]:
# Check number of unique shopid
# (should be the total number of rows for the submission file)
len(set(dataset.shopid.tolist()))

18770

# Data Preparation

In [6]:
# Convert to datetime
dataset.event_time = pd.to_datetime(
    dataset.event_time, 
    format='%Y-%m-%d %H:%M:%S', 
    errors='coerce'
)

In [7]:
# Sort the dataset by shopid and event time
dataset = dataset.sort_values(by=['shopid', 'event_time'])

In [8]:
# Show the dataset
dataset.head()

,orderid,shopid,userid,event_time
59,31086409141107,10009,196962305,2019-12-27 03:06:50
76187,31144571933461,10051,2854032,2019-12-27 19:16:11
9055,31254979546679,10051,48600461,2019-12-29 01:56:19
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32


# Analysis

In [9]:
# Determine order brushing
counter = 0
brushing = []
for idx, row in dataset.iterrows():

    # Get all required info
    orderid = row[0]
    shopid = row[1]
    userid = row[2]
    event_time = row[3]
    
    # Get one hour limitation time
    end_time = row[3] + timedelta(hours=1)

    # Get all orders within one hour
    subset = dataset.loc[
        (dataset.shopid == shopid) 
        & (dataset.event_time >= event_time)
        & (dataset.event_time <= end_time)
    ]

    # Calculate concentrate rate
    users = list(np.unique(subset.userid))
    rate = subset.shape[0] / len(users)

    # Get the user with the highest proportion
    if rate >= 3:
        
        # Get total orders per userid
        subset_agg = subset[['userid', 'orderid']] \
            .groupby('userid') \
            .count() \
            .reset_index()
        max_order = subset_agg.orderid.max()
        
        # Get userid with max number of orders
        users = subset_agg \
            .loc[subset_agg.orderid == max_order, 'userid'] \
            .unique()
        
        # Get orders from suspicious userid
        orders = subset \
            .loc[subset.userid.isin(users), ['shopid', 'userid', 'orderid']] \
            .values \
            .tolist()
        brushing = brushing + orders
    else:
        # Check the first three orders
        if subset.shape[0] >= 4:
            
            # Check if only from the same userid
            unique_id = len(subset.iloc[:3, 2].unique())
            if unique_id == 1:
                
                # Compare the time event with the next and previous orders
                end_time = subset.iloc[3, 3]
                previous = dataset.loc[ \
                    (dataset.shopid==shopid) \
                    & (dataset.event_time < event_time) \
                , :]
                if previous.shape[0] > 0:
                    start_time = previous.iloc[-1, 3]
                    delta = end_time - start_time
                
                # Get orders from suspicious userid
                if previous.shape[0] == 0 or delta > timedelta(hours=1):
                    orders = subset \
                        .iloc[:3, [1, 2, 0]] \
                        .values \
                        .tolist()
                    brushing = brushing + orders

    # Show log
    counter = counter + 1
    if counter % 10000 == 0:
        print(str(counter), 'rows checked')

10000 rows checked
20000 rows checked
30000 rows checked
40000 rows checked
50000 rows checked
60000 rows checked
70000 rows checked
80000 rows checked
90000 rows checked
100000 rows checked
110000 rows checked
120000 rows checked
130000 rows checked
140000 rows checked
150000 rows checked
160000 rows checked
170000 rows checked
180000 rows checked
190000 rows checked
200000 rows checked
210000 rows checked
220000 rows checked


In [10]:
# Remove duplication and aggregate
brushing = pd.DataFrame(brushing, columns=['shopid', 'userid', 'orderid'])
brushing = brushing.drop_duplicates()
brushing = brushing.groupby(['shopid', 'userid']).count().reset_index()

In [11]:
# Get the suspicious buyers
highest = brushing[['shopid', 'orderid']].groupby(['shopid']).max()
suspicious = pd.merge(brushing, highest, on=['shopid', 'orderid'], how='inner')

In [12]:
# Form the final result
suspicious = suspicious.sort_values(by=['shopid', 'orderid'])
suspicious.userid = suspicious.userid.astype('str')
result = suspicious.groupby('shopid')['userid'].apply('&'.join).reset_index()

In [13]:
# Merge with shopid without brushing orders
shopid_all = list(set(dataset.shopid.tolist()))
shopid_clean = pd.DataFrame(shopid_all, columns=['shopid'])
result = pd.merge(shopid_clean, result, on='shopid', how='left')
result = result.fillna(0)

In [14]:
# Export to csv
result.to_csv('submission.csv', index=False)